In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Loading various Datasets

### timeseries

In [25]:
ts_df = pd.read_csv('https://api.covid19india.org/csv/latest/case_time_series.csv')
ts_df.shape

(444, 8)

In [26]:
total_cases_df = ts_df[['Date', 'Date_YMD', 'Total Confirmed', 'Total Recovered', 'Total Deceased']]
daily_cases_df = ts_df[['Date', 'Date_YMD', 'Daily Confirmed', 'Daily Recovered', 'Daily Deceased']]

In [27]:
total_cases_df.tail()

,Date,Date_YMD,Total Confirmed,Total Recovered,Total Deceased
439,13 April 2021,2021-04-13,13871431,12332564,171524
440,14 April 2021,2021-04-14,14071015,12425989,172562
441,15 April 2021,2021-04-15,14287843,12543886,173746
442,16 April 2021,2021-04-16,14521845,12666772,175084
443,17 April 2021,2021-04-17,14782740,12804981,176582


### Statewise

In [12]:
state_df = pd.read_csv("https://api.covid19india.org/csv/latest/state_wise.csv")
state_daily_df = pd.read_csv("https://api.covid19india.org/csv/latest/state_wise_daily.csv")
state_wise_test_df = pd.read_csv("https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv")
st_df = pd.read_csv("https://api.covid19india.org/csv/latest/states.csv")

In [14]:
state_df.head()

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,Migrated_Other,State_code,Delta_Confirmed,Delta_Recovered,Delta_Deaths,State_Notes
0,Total,14792362,12806991,177193,1801951,18/4/2021 13:28:55,6227,TT,9539,1844,20,NaN
1,Maharashtra,3770707,3061174,59970,647933,17/04/2021 20:41:54,1630,MH,0,0,0,"[Dec 16]:10,218 duplicate cases & other state ..."
2,Kerala,1221168,1135921,4905,80016,18/04/2021 13:28:56,326,KL,0,0,0,Mahe native who expired in Kannur included in ...
3,Karnataka,1141998,1009549,13270,119160,17/04/2021 19:26:58,19,KA,0,0,0,NaN
4,Andhra Pradesh,955455,907598,7388,40469,17/04/2021 18:17:58,0,AP,0,0,0,NaN


In [19]:
st_df.tail()

,Date,State,Confirmed,Recovered,Deceased,Other,Tested
14427,2021-04-18,Tripura,33999,33137,391,23,662606.0
14428,2021-04-18,India,14792362,12806991,177193,6227,266538416.0
14429,2021-04-18,Uttar Pradesh,821054,641292,9703,0,38029865.0
14430,2021-04-18,Uttarakhand,121403,101659,1856,2502,3300338.0
14431,2021-04-18,West Bengal,651508,595668,10540,0,9762086.0


In [20]:
state_daily_df.tail()

,Date,Date_YMD,Status,TT,AN,AP,AR,AS,BR,CH,CT,DN,DD,DL,GA,GJ,HR,HP,JK,JH,KA,KL,LA,LD,MP,MH,MN,ML,MZ,NL,OR,PY,PB,RJ,SK,TN,TG,TR,UP,UT,WB,UN
1195,16-Apr-21,2021-04-16,Recovered,122886,28,2194,2,113,1853,428,12240,25,0,12649,282,3387,2958,484,562,1393,4027,3792,47,11,7496,45335,7,12,5,2,952,220,3129,2791,10,4920,1198,7,6429,1080,2818,0
1196,16-Apr-21,2021-04-16,Deceased,1338,0,20,0,4,13,1,138,0,0,141,6,94,20,9,2,56,78,21,0,0,60,398,0,1,0,1,3,3,50,31,0,33,9,0,103,17,26,0
1197,17-Apr-21,2021-04-17,Confirmed,260895,42,7224,19,876,7870,431,16083,108,0,24375,762,9541,7717,1392,1145,3838,17489,13835,362,106,11269,67123,39,95,62,32,3144,715,4431,9046,72,9344,4446,58,27334,2757,7713,0
1198,17-Apr-21,2021-04-17,Recovered,138209,36,2332,6,149,1804,438,9828,25,0,15414,436,3783,2944,656,852,1234,5564,3654,53,5,6497,56783,14,11,1,0,1303,198,2615,2823,4,5263,1414,11,7831,802,3426,0
1199,17-Apr-21,2021-04-17,Deceased,1498,1,15,0,2,34,2,158,0,0,167,4,97,32,12,3,30,80,27,1,0,66,419,0,0,0,0,4,3,62,37,0,39,12,0,120,37,34,0


In [24]:
state_wise_test_df.shape

(12416, 55)

### Districts

In [28]:
dist_df = pd.read_csv('https://api.covid19india.org/csv/latest/districts.csv')

In [30]:
dist_df.tail()

,Date,State,District,Confirmed,Recovered,Deceased,Other,Tested
229124,2021-04-18,West Bengal,Purba Bardhaman,14196,13051,106,0,NaN
229125,2021-04-18,West Bengal,Purba Medinipur,21916,20909,281,0,NaN
229126,2021-04-18,West Bengal,Purulia,8740,7691,52,0,NaN
229127,2021-04-18,West Bengal,South 24 Parganas,41890,38393,727,0,NaN
229128,2021-04-18,West Bengal,Uttar Dinajpur,7548,6941,78,0,NaN


In [33]:
dd = pd.read_csv('http://api.covid19india.org/csv/latest/cowin_vaccine_data_statewise.csv')


,Updated On,State,Total Individuals Registered,Total Sessions Conducted,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,Total Individuals Vaccinated,Total Doses Administered
0,16/01/2021,India,6943622.0,3455.0,2957.0,48276.0,0.0,23757.0,24517.0,2.0,579.0,47697.0,48276.0,48276.0
1,17/01/2021,India,7119567.0,8532.0,4954.0,58604.0,0.0,27348.0,31252.0,4.0,635.0,57969.0,58604.0,58604.0
2,18/01/2021,India,7150780.0,13611.0,6583.0,99449.0,0.0,41361.0,58083.0,5.0,1299.0,98150.0,99449.0,99449.0
3,19/01/2021,India,7423057.0,17855.0,7951.0,195525.0,0.0,81901.0,113613.0,11.0,3017.0,192508.0,195525.0,195525.0
4,20/01/2021,India,7555207.0,25472.0,10504.0,251280.0,0.0,98111.0,153145.0,24.0,3946.0,247334.0,251280.0,251280.0


In [35]:
dd[dd['State'] == 'Maharashtra']

,Updated On,State,Total Individuals Registered,Total Sessions Conducted,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,Total Individuals Vaccinated,Total Doses Administered
1953,16/01/2021,Maharashtra,812649.0,179.0,174.0,5726.0,0.0,3668.0,2057.0,1.0,85.0,5641.0,5726.0,5726.0
1954,17/01/2021,Maharashtra,814714.0,269.0,216.0,6521.0,0.0,3953.0,2566.0,2.0,94.0,6427.0,6521.0,6521.0
1955,18/01/2021,Maharashtra,811257.0,772.0,320.0,6151.0,0.0,3569.0,2581.0,1.0,105.0,6046.0,6151.0,6151.0
1956,19/01/2021,Maharashtra,850966.0,1196.0,340.0,13699.0,0.0,6328.0,7367.0,4.0,214.0,13485.0,13699.0,13699.0
1957,20/01/2021,Maharashtra,870296.0,1547.0,347.0,23880.0,0.0,9658.0,14205.0,17.0,439.0,23441.0,23880.0,23880.0
1958,21/01/2021,Maharashtra,880118.0,1820.0,357.0,24148.0,0.0,9784.0,14347.0,17.0,443.0,23705.0,24148.0,24148.0
1959,22/01/2021,Maharashtra,902635.0,2012.0,374.0,44369.0,0.0,16546.0,27802.0,21.0,840.0,43529.0,44369.0,44369.0
1960,23/01/2021,Maharashtra,921922.0,2435.0,483.0,69295.0,0.0,25333.0,43937.0,25.0,1156.0,68139.0,69295.0,69295.0
1961,24/01/2021,Maharashtra,925794.0,2756.0,581.0,70196.0,0.0,25661.0,44510.0,25.0,1157.0,69039.0,70196.0,70196.0
1962,25/01/2021,Maharashtra,950534.0,3170.0,617.0,107073.0,0.0,38512.0,68527.0,34.0,1433.0,105640.0,107073.0,107073.0
